In [2]:
import numpy as np
from sklearn.datasets import make_blobs
from dataclasses import dataclass
import sklearn
from scipy.stats import norm
from sklearn.model_selection import train_test_split

In [1]:

# make_blobs?


In [3]:
X,y = make_blobs(n_samples = 10000, n_features = 2, centers = 2)

In [4]:
print(X[:5])

[[-0.34427316  8.56196103]
 [-1.07578503  8.15163062]
 [-1.04786781  8.35811957]
 [ 1.95338238  0.19797266]
 [-2.42732815  9.01584035]]


In [5]:
print(y[:5])

[0 0 0 1 0]


Points about NB:
- Generative model
- Non-parametric model
- It assumes a gaussian distribution for every column of each class
- It naively assumes that columns are independent
  

- It doesn't have any hyper-parameters

What do we need:
- Prior Distribution
- Likelihood
- Posterior ~ Likelihood * Prior
- Datasplit

Gaussian_PDF = 1/(sqrt(2 * pi * var) * (exp(-(x_i - mean)**2/2var))
p(y_i = k | x_i) ~ P(x_i_0 | y = k) * P(x_i_1 | y = k) * P(y=k)


In [6]:
@dataclass
class GaussianNaiveBayes:
    # def __init__(self, X, y):
    #     self.X = X
    #     self.y = y
    X: np.array
    y: np.array

    def __post_init__(self): #series of steps that you want your class object to execute/ initiate as soon as someone initiates the class without a need to ask the user
        self.dataSplit()
        
        self.X0_train = self.X_train[self.y_train == 0]
        self.X1_train = self.X_train[self.y_train == 1]
        self.y0_train = self.y_train[self.y_train == 0]
        self.y1_train = self.y_train[self.y_train == 1]

        self.fit()
        self.predict()
    #a new thing taught about data class

    def dataSplit(self):
        self.X_train, self.X_test, self.y_train, self.y_test = sklearn.model_selection.train_test_split(self.X, self.y, test_size = 0.3, shuffle = True)

    def fit_distribution(self, x): #we are passing one column at a time
        mean = np.mean(x)
        std = np.std(x)
        dist = norm(mean, std)
        return dist 

    def posterior(self, x, prior, dist_col1, dist_col2):
        return prior * dist_col1.pdf(x[0]) * dist_col2.pdf(x[1]) # dist_col1.pdf(x[0]) is the likelihood of that point in that distribution
        
    def fit(self):
        self.prior_y0 = len(self.y0_train) / len(self.y)
        self.prior_y1 = len(self.y1_train) / len(self.y)

        #Distribution of every column in each class
        
        self.dist_X_0_0 = self.fit_distribution(self.X0_train[:,0])
        self.dist_X_0_1 = self.fit_distribution(self.X0_train[:,1])

        self.dist_X_1_0 = self.fit_distribution(self.X1_train[:,0])
        self.dist_X_1_1 = self.fit_distribution(self.X1_train[:,1])

    def predict(self):

        self.error_count = 0

        for sample, target in zip(self.X_test, self.y_test):
            py0 = self.posterior(sample, self.prior_y0, self.dist_X_0_0, self.dist_X_0_1)
            py1 = self.posterior(sample, self.prior_y1, self.dist_X_1_0, self.dist_X_1_1)

            print('P(y=0 | %s) = %3.f' % (sample, py0 * 100))
            print('P(y=1 | %s) = %3.f' % (sample, py1 * 100))

            # if np.argmax([py0,py1]) != target:
            #     self.error_count += 1
        
            print(" Model predicted class {} and the truth was: {} \n".format(np.argmax([py0,py1]), target))

        # print(self.error_count)
        # print(len(self.X))

In [7]:
nb = GaussianNaiveBayes(X,y)

P(y=0 | [-1.76387493  9.46455233]) =   2
P(y=1 | [-1.76387493  9.46455233]) =   0
 Model predicted class 0 and the truth was: 0 

P(y=0 | [ 3.40813788 -0.53119548]) =   0
P(y=1 | [ 3.40813788 -0.53119548]) =   4
 Model predicted class 1 and the truth was: 1 

P(y=0 | [0.87854427 7.71983373]) =   1
P(y=1 | [0.87854427 7.71983373]) =   0
 Model predicted class 0 and the truth was: 0 

P(y=0 | [-0.11454347  9.04524999]) =   3
P(y=1 | [-0.11454347  9.04524999]) =   0
 Model predicted class 0 and the truth was: 0 

P(y=0 | [-1.70028957  7.33738074]) =   2
P(y=1 | [-1.70028957  7.33738074]) =   0
 Model predicted class 0 and the truth was: 0 

P(y=0 | [-0.18804647  9.54412137]) =   2
P(y=1 | [-0.18804647  9.54412137]) =   0
 Model predicted class 0 and the truth was: 0 

P(y=0 | [ 2.17638123 -0.0563021 ]) =   0
P(y=1 | [ 2.17638123 -0.0563021 ]) =   5
 Model predicted class 1 and the truth was: 1 

P(y=0 | [-0.99691349  8.15967497]) =   5
P(y=1 | [-0.99691349  8.15967497]) =   0
 Model predi